# Galaxy Selector MOC

**Designed to work with Multi-Ordering-Sky-Maps and both burst (i.e. no distance) and normal LVK maps**



Select the galaxies to observe by WWFI in this simple manner:
- get the LIGO event data
- read the LIGO event data into the notebook, extract some information
- read the DESI database into the notebook, "clean" the data
- get the 99% credible regions for the event, add them as a column to the data
- only keep data within the 99% region
- calculate all the luminosities (& more)
- rank them by luminosity (for now just print the TARGETID), 3D/2D localization and with a luminosity-distance 2D dependant counterpart likelihood

(some) Sources: 
- https://iopscience.iop.org/article/10.3847/0067-0049/226/1/10
- https://emfollow.docs.ligo.org/userguide/tutorial/skymaps.html
- https://emfollow.docs.ligo.org/userguide/tutorial/multiorder_skymaps.html
- https://arxiv.org/pdf/1710.05452.pdf

## Import all libraries, load GW data and DESI data and extract basic info

### imports, GW data loading

In [1]:
import numpy as np
from scipy.stats import norm
import scipy as sc
from scipy.integrate import quad


from astropy.io import fits, ascii
from astropy.table import Table, hstack, QTable
from astropy import table
from astropy.coordinates import SkyCoord, EarthLocation, AltAz

import astropy_healpix as ah


import psycopg2

from scipy.stats import norm

from astropy.cosmology import FlatLambdaCDM
from astropy.cosmology import Planck18, z_at_value
from astropy.coordinates import Distance, SkyCoord
from astropy import units as u

import astropy.constants as asc

import matplotlib.pyplot as plt

import requests as rq

import gasel as gs

from datetime import datetime
from datetime import timedelta
from datetime import date
from suntimes import SunTimes
import pytz

In [65]:
# Get DESI data
print("(to get DESI data) you should run scp dgruen@perlmutter.nersc.gov:/pscratch/sd/j/jgassert/good_z_data.fits /project/ls-gruen/ligodesi/")

# specify the superevent name:
superevent_name = "S200225q"

# pipeline - bayestar is preferred if available, otherwise maybe use olib? bilby? cwb? whatever these mean
pipeline_type = ""

if pipeline_type == "cwb":
    # bursts probably only for very nearby core-collapse or Andromeda events, maybe cut at 1Mpc or something of that order? -> check if there is a very nearby galaxy
        # think about which pipeline detected it!
    # or very massive bbh but then hope is that cbc would detect as well
    
    modeled_search = False
else:
    modeled_search = True

# storage directory
# storage = "/project/ls-gruen/ligodesi/"
# storage_desi_data = storage
storage = "/global/homes/j/jgassert/ligo-desi-gw-follow-up/Galaxy Selection/Files/" # Julius on NERSC
storage_desi_data = "/pscratch/sd/j/jgassert/"

(to get DESI data) you should run scp dgruen@perlmutter.nersc.gov:/pscratch/sd/j/jgassert/good_z_data.fits /project/ls-gruen/ligodesi/


In [66]:
path = storage+f"{superevent_name}{pipeline_type}.LALInference.fits.gz" # 5min after alert this should be there

try:
    skymap = QTable.read(path)
    print("read skymap from local file")
    
except:
    download_url = f"https://gracedb.ligo.org/api/superevents/{superevent_name}/files/{pipeline_type}.LALInference.fits.gz"
    save_path = storage+f"{superevent_name}{pipeline_type}.LALInference.fits.gz"
    print("no local skymap...")
    print(f"...will download LIGO event file from superevent {superevent_name}, corresponding file URL is {download_url} and save it in .../Files/")
    
    response = rq.get(download_url, allow_redirects = True)
    print("HTTP status",response.status_code)
    if response.status_code == 200:
        with open(save_path, 'wb') as f:
            f.write(response.content)
            
        skymap = QTable.read(path)

no local skymap...
...will download LIGO event file from superevent S200225q, corresponding file URL is https://gracedb.ligo.org/api/superevents/S200225q/files/.LALInference.fits.gz and save it in .../Files/
HTTP status 404


Specifiy the path where the data is located and then load the data (this loads everything, just have a look)

In [67]:
skymap[:5]

UNIQ,PROBDENSITY,DISTMU,DISTSIGMA,DISTNORM
,1 / sr,Mpc,Mpc,1 / Mpc2
int64,float64,float64,float64,float64
1024,2.0628782133841405e-12,-1012.2049520497791,376.82374575948757,0.011095199813917983
1025,8.27362108428298e-12,166.23094482047838,153.23732378694845,2.014040931456908e-05
1026,1.4736272931268072e-12,inf,1.0,0.0
1027,6.780677710008301e-12,102.07357208021745,116.54675869191472,4.412607201084553e-05
1028,5.79319209390381e-11,78.61969670578218,210.35830952485827,2.582873160902654e-05


### Get basic values from GW event

We now extract some basic information  from the healpix data:
- ipix_max: pixel with highest likelihood
- npix: total number of pixels
- ra, dec: ra and dec of the pixel with the highest likelihood

Then we open the full fits file and extract some more basic info from the header:
- dist_mean: mean distance of the GW event
- dist_std: error of the distance

In [68]:
# most probable sky location
i = np.argmax(skymap["PROBDENSITY"])
uniq = skymap[i]["UNIQ"]
print("UNIQ ID of highest prob sky localization: ", uniq)

UNIQ ID of highest prob sky localization:  1494052


In [69]:
# calculate the most probable pixel, convert to RA and dec
level_max, ipix_max = ah.uniq_to_level_ipix(uniq)
nside = ah.level_to_nside(level_max)
ra, dec = ah.healpix_to_lonlat(ipix_max, nside, order = "nested")
print("Highest probability at (RA, dec): ", ra.deg, dec.deg, "with prob [1/deg^2] of: ", skymap[i]['PROBDENSITY'].to_value(u.deg**-2), "and UNIQ ID: ", skymap[i]["UNIQ"])

Highest probability at (RA, dec):  108.67346938775509 48.82520674420733 with prob [1/deg^2] of:  0.005534703004111113 and UNIQ ID:  1494052


Now open the fits file and read basics info from the header

In [70]:
if modeled_search:
    fits_gw = fits.open(path)
    header = fits_gw[1].header
    dist_mean = header["DISTMEAN"]
    dist_std = header["DISTSTD"]
    print("The dist_mean and dist_std values: ", dist_mean*u.Mpc, dist_std*u.Mpc)
    most_likely_z = z_at_value(Planck18.luminosity_distance, dist_mean*u.Mpc, zmax = 5)
    most_likely_z_std = z_at_value(Planck18.luminosity_distance, dist_std*u.Mpc, zmax = 5)
    print(f"This corresponds to the most likely z {most_likely_z} and its error {most_likely_z_std}")
else:
    print("As this is an unmodeled search there is no distance available")

FileNotFoundError: [Errno 2] No such file or directory: '/global/homes/j/jgassert/ligo-desi-gw-follow-up/Galaxy Selection/Files/S200225q.LALInference.fits.gz'

### Get DESI data

Load the full daily data from the file, which only includes good redshift objects

In [ ]:
follow_up_others = False

In [ ]:
if follow_up_others:
    center_ra = 235.9839
    center_dec = 15.2248
    radius = 1/360
    z_others = 0
    z_others_std = 0.4
    
    sigma_accuracy = 3
    
    data = gs.db_doall(center_ra, center_dec, radius)
    
    #ii = data["Z"]-sigma_accuracy*data["ZERR"]<z_others+sigma_accuracy*z_others_std
    #ii &= data["Z"]+sigma_accuracy*data["ZERR"]>z_others-sigma_accuracy*z_others_std
    
    #data = data[ii]
    
    
else:
    data = Table.read(storage_desi_data+"good_z_data.fits")

In [ ]:
data[:5]

## Do all the calculations, selections,...

- turn the daily DESI data into a Table
- do some basic data selection stuff: only keep good + positive redshifts, only positive values of r-band flux and eliminate duplicates (right now simply takes the first entry)
- add the ipix (healpix) pixel value for each target
- calculate the probability for each target
- add these values to the Table ("PROB"), select only targets within the 99% credible region (i.e. with a certainty of 99% the GW event is inside this region)
- calculate the distances and its errors from the redshift, add to the data table
- calculate the 3D probability just like here: https://iopscience.iop.org/article/10.3847/0067-0049/226/1/10 (§4) and add these values to the table
- sort the table by its 3D probability (descending)
- calculate absolute and apparent magnitude, luminosity and add to table

The final step is to include the calculation of the most likely host galaxy from https://arxiv.org/pdf/1710.05452.pdf (GW170817); these values are then also added to the table as "P_GAL"

In [ ]:
data[:5]

### data clean up

In [ ]:
#data = data[data["Z"]> 0] # this migh have 2 reasons
#data = data[data["ZWARN"] == 0]

In [ ]:
#data = data[data['FLUX_Z']>0]
# data = table.unique(data, keys = "TARGETID")

In [ ]:
data = data[data["TARGETID"]>0]

In [ ]:
print(f"We have {len(data)} objects in the full catalog")

We should also get rid of stars, since they wont be the host of a GW event

In [ ]:
data = data[data["SPECTYPE"] != "STAR"]

### Calculate the healpix id for every object in the catalog and its probability; append this data to the Table

In [ ]:
max_level = 29 # highest possible HEALPix resolution that can be represented in a 64-bit signed integer
max_nside = ah.level_to_nside(max_level)
level, ipix = ah.uniq_to_level_ipix(skymap["UNIQ"])

index = ipix*(2**(max_level-level))**2

sorter = np.argsort(index)

# this is the NESTED pixel index of the target sky location, NOT THE UNIQ ID
match_ipix = ah.lonlat_to_healpix(data["TARGET_RA"]*u.deg, data["TARGET_DEC"]*u.deg, max_nside, order='nested')

# here we get the pixel index (i.e. where the entry is in the table, also NOT THE UNIQ ID) of each target
sorter_i = sorter[np.searchsorted(index, match_ipix, side='right', sorter=sorter) - 1]
probdensity = skymap[sorter_i]['PROBDENSITY'].to_value(u.deg**-2)

# now we calculate the UNIQ ID for all our targets:

uniq = skymap["UNIQ"][sorter_i]
#to make things easier in future, I will now also append the UNIQ pixels DISTMU, DISTSIGMA and DISTNORM values. However, be careful, since these DO NOT CORRESPOND THE TARGETS IN DESI DIRECTLY!

if modeled_search:
    distmu = skymap[sorter_i]["DISTMU"]
    distsigma = skymap[sorter_i]["DISTSIGMA"]
    distnorm = skymap[sorter_i]["DISTNORM"]


In [ ]:
# len(UNIQ), len(data), len(skymap)

In [ ]:
#data = data.keep_columns(["TARGETID", "TARGET_RA", "TARGET_DEC", "TILEID", "NIGHT", "Z", "ZERR", "ZWARN", "DELTACHI2", "FLUX_Z", "BGS_TARGET", "EBV", "SERSIC", "MWS_TARGET", "FILENAME"])
                          

In [ ]:
if modeled_search:
    data = hstack([data, Table({"UNIQ": uniq}), Table({"PROBDENSITY": probdensity}), Table({"DISTMU": distmu}), Table({"DISTSIGMA": distsigma}), Table({"DISTNORM": distnorm})])
else: 
    data = hstack([data, Table({"UNIQ": uniq}), Table({"PROBDENSITY": probdensity})])
    print(f"The table will not contain any distance info since there is none available")

In [ ]:
data[:5]

In [ ]:
print(f"The highest and lowest probability values in our catalog are: {np.max(probdensity)} and {np.min(probdensity)}")

### Sort the skymap table and get 99% credible region

In [ ]:
# copy the table
skymap_sort = Table(skymap, copy = True)

In [ ]:
# check whether copy worked:
# skymap_sort, skymap
# np.max(skymap["UNIQ"])

In [ ]:
# now sort the skymap by its probability (descending)
skymap_sort.sort('PROBDENSITY', reverse=True)

In [ ]:
# this shows the first entries sorted be their probability
skymap_sort[:5]

In [ ]:
# now get 99% credible region area

level, ipix = ah.uniq_to_level_ipix(skymap_sort['UNIQ'])
pixel_area = ah.nside_to_pixel_area(ah.level_to_nside(level)) # this is the area each pixel contains

# calculate the probability per pixel (careful: since the pixels differ in size, this is not neceseraliy a useful quantity
prob = pixel_area * skymap_sort['PROBDENSITY']

# calc cumsum of probabilities and get index of pixel that sums up to 0.99
cumprob = np.cumsum(prob)
i = cumprob.searchsorted(0.99)

# print area
area_99 = pixel_area[:i].sum()
area_99_deg = area_99.to_value(u.deg**2)
print(f"The 99% search area is {area_99_deg}°large")

### can be deleted, just tried things out

In [ ]:
skymap_sort = hstack([skymap_sort, Table({"PROB": prob})])

In [ ]:
np.max(prob)

In [ ]:
skymap_sort[:5]

In [ ]:
skymap_prob = Table(skymap_sort, copy = True)

In [ ]:
skymap_prob.sort("PROB", reverse = True)

In [ ]:
skymap_prob[:5]

### Compare Tables, only leave 99% credible region in DESI data table

Now we can get a Table that only contains the 99% credible region. From there we only leave the matching objects in the DESI data

In [ ]:
# we simply select only the entries that are within the 99% credible region
skymap_99 = skymap_sort[:i]

In [ ]:
print("This is the number of UNIQ ID of objects which should lie within the 99% credible region (NOT THE NUMBER OF OBJECTS, as there can be multiple objects per UNIQ ID):", len(np.intersect1d(skymap_99["UNIQ"], data["UNIQ"])))

In [ ]:
mask = np.in1d(data["UNIQ"], skymap_99["UNIQ"])
data_99 = data[mask]

In [ ]:
print(f"We now have {len(data_99)} objects in the 99% credible region")
print("Check if the region from where the galaxies have been selected makes any sense (RA; DEC): ", np.min(data_99["TARGET_RA"]), np.max(data_99["TARGET_RA"]), np.min(data_99["TARGET_DEC"]), np.max(data_99["TARGET_DEC"]))

max_probdens_in_desi = np.max(data_99["PROBDENSITY"])
max_probdens_in_ligo = np.max(skymap["PROBDENSITY"]).to_value(u.deg**-2)
print(f"We can also check whether we have at least one galaxy in the UNIQ pixel with the highest probability from LIGO...")
print(f"This is {np.allclose(max_probdens_in_desi, max_probdens_in_ligo)}")

In [ ]:
data_99.sort('PROBDENSITY', reverse=True)

In [ ]:
data_99[:2]

In [ ]:
if modeled_search:
    print("We have now performed all the necessary steps to get the localization selection in 2D, i.e. the angle on the sky")
    min_z = np.min(data_99["Z"])
    max_z = np.max(data_99["Z"])
    print(f"However, we have not used the distance data at all. Therefore our redshifts range from {min_z} to {max_z}")
else:
    print(f"Usually we would be performing steps to cut out objects that don't meet the z-range/distance from LIGO, since there is no distance, we - at least for now include everything!")

### Calculate all the distances of the objects from the redshifts

We want to be conservative in estimating the distance from redshifts, therefore we will include different cosmologies.

Our cosmologies include:
- H0 = 68.20 ± 0.81 km s−1Mpc−1, from eBOSS https://arxiv.org/pdf/2007.08991.pdf
- DES Y3 https://arxiv.org/abs/2105.13549
- SH0ES https://pantheonplussh0es.github.io/

Now we calculate the arithmetic mean of our distances and its standard deviation:
$$distances_{mean} = \frac{dist(z_{cosmo_{max}})+dist(z_{cosmo_{min}})}{2}$$
and accordingly the standard deviation (n=2):
$$\sigma = \sqrt{\frac{1}{n-1}\sum\left(dist_i-distances_{mean}\right)^2}$$


In [ ]:
#distances = Distance(z=data_99["Z"], cosmology=Planck18)
#distances_err = Distance(z=data_99["ZERR"], cosmology=Planck18)

In [ ]:
if modeled_search:
    # first define the cosmological maximum and minimum parameters
    omega_M_max = 0.339+0.032 #(from DES)
    omega_M_min = 0.299-2*0.016 #from eBOSS

    H_0_max = 73.04+2*1.04 #from SH0ES
    H_0_min = 67.4-2*0.5 #from Planck
    
    # then calculate min and max luminosity distance from the redshifts
    cosmo_min_dist = FlatLambdaCDM(H0=H_0_max, Om0=omega_M_max)
    cosmo_max_dist = FlatLambdaCDM(H0=H_0_min, Om0=omega_M_min)

    dists_max = Distance(z=data_99["Z"], cosmology=cosmo_max_dist)
    dists_min = Distance(z=data_99["Z"], cosmology=cosmo_min_dist)
    
    # and of course also calculate the maximum and minimum error from the redshfit uncertainty
    dists_err_min = Distance(z=data_99["ZERR"], cosmology=cosmo_max_dist)
    dists_err_max = Distance(z=data_99["ZERR"], cosmology=cosmo_min_dist)
    
    # now we get the mean distance and its error
    dists_mean = (dists_max+dists_min)/2
    dists_err_mean = (dists_err_max+dists_err_min)/2  # this is our sigma for the redshift error
    
    # calculate the stdev of the distance depending on the cosmology
    dists_mean_stddev = np.sqrt((dists_max-dists_mean)**2+(dists_min-dists_mean)**2)
    
    # now stack all the data
    data_99 = hstack([data_99, Table({"DIST_Z_MEAN": dists_mean}), Table({"DIST_Z_ERR": dists_err_mean}), Table({"DIST_Z_COSMO_SIGMA": dists_mean_stddev})])

In [ ]:
# dists_mean[:5], dists_mean_stddev[:5], dists_mean_stddev[:5]/dists_mean[:5], np.mean(dists_mean_stddev/dists_mean), np.max(dists_mean_stddev/dists_mean), np.min(dists_mean_stddev/dists_mean)


In [ ]:
data_99[:5]

### Lets check how much the distance measurement changes in the 99% credible region of LIGO and compare to our DESI data

We should see a lot more data in DESI (since we haven't done any redshift selection and there can be multiple objects per GW-pixel). Be carfeul, we are comparing actual object counts with the number of pixels.


We still see, that most of the GW localization is roughly between 780Mpc and 950Mpc. DESI definitely has a most of its data in this range as well.

In [ ]:
if modeled_search:
    plt_skymap = skymap_99[skymap_99["DISTMU"]<np.inf]

    dist_max_99 = np.max(plt_skymap["DISTMU"])*u.Mpc
    dist_min_99 = np.min(plt_skymap["DISTMU"])*u.Mpc
    print(f"Our LIGO distances range from {dist_min_99} to {dist_max_99}")
    print("Let's visualize the distance distribution of the LIGO event in the 99% credible region")

    fig, ax = plt.subplots(ncols = 2, figsize = (15,8))


    ax[0].hist(data_99["DIST_Z_MEAN"], bins = "auto", color = "orange", label = "DESI data")
    ax[1].hist(plt_skymap["DISTMU"], bins = "auto", color = "blue", label = "LIGO data")


    for axis in ax:
        axis.set_xlabel("distance bin (Mpc)")
        axis.set_xlim(dist_min_99/u.Mpc, dist_max_99/u.Mpc)
        axis.legend()

    ax[0].set_ylabel("object count")
    ax[1].set_ylabel("pixel count")

    plt.show()

In [ ]:
len(skymap_99), len(data_99)

### To reduce our data even more, lets cut out the unreasonably far away objects

We simply take maximum and minimum distance we get from the LIGO skymap (i.e. DISTMU+-DISTERR) and check, whether every galaxy in that pixel falls inside this range.

*Note: We do this only for our own GW-follow ups*

In [ ]:
if follow_up_others == False and modeled_search == True:
    z_selection_mask = data_99["DIST_Z_MEAN"] - 2*data_99["DIST_Z_ERR"] - 2*data_99["DIST_Z_COSMO_SIGMA"] <  data_99["DISTMU"] + 2*data_99["DISTSIGMA"]
    z_selection_mask &= data_99["DIST_Z_MEAN"] + 2*data_99["DIST_Z_ERR"] + 2*data_99["DIST_Z_COSMO_SIGMA"] >  data_99["DISTMU"] - 2*data_99["DISTSIGMA"]

    data_99_z = data_99[z_selection_mask]

    remaining_obj = np.size(np.where(z_selection_mask == True))
    deleted_obj = np.size(np.where(z_selection_mask == False))

    print(f"The z-range selection has deleted {deleted_obj} objects from originally {deleted_obj+remaining_obj} in the data set")
    print(f"Therefore, we have {remaining_obj} objects left in the search region")

    highest_prob_dens = np.max(data_99_z["PROBDENSITY"])
    print(f"Our highest remaing probability density is {highest_prob_dens} compared to the originally highest prob density of {max_probdens_in_ligo}")
    print(f"The difference in prob density is {max_probdens_in_ligo-highest_prob_dens}")

    
else:
    data_99_z = data_99
    remaining_obj = len(data_99_z)

In [ ]:
data_99_z[:5]

### get the apparent, absolute magnitudes and luminosities

in order to do this: flux > 0 selection  and gal ext correction

In [ ]:
# get the apparent mags
orig_len = len(data_99_z)
data_99_z = data_99_z[data_99_z["FLUX_R"] > 0] # flux selection (only ones above 0)
print(f"had to cut out {len(data_99_z)-orig_len} objects due to bad fluxes")
data_99_z["MW_TRANSMISSION"] = gs.mw_transmission_from_data_table(data = data_99_z, band = "r")
#data_99_z["FLUX_R"] = data_99_z["FLUX_R"]/data_99_z["MW_TRANSMISSION"]

app_mag_r = gs.app_mag(data_99_z["FLUX_R"], mw_transmission = data_99_z["MW_TRANSMISSION"])
flux_selection_cut = len(data_99_z)

# get the absolute mags
abs_mag_r = gs.abs_mag(app_mag_r, data_99_z["Z"], 0,0,0)

# get the luminosities
lums_r = gs.lum(abs_mag_r, band = "r")

In [ ]:
data_f = hstack([data_99_z, Table({"APP_MAG_R": app_mag_r}), Table({"ABS_MAG_R": abs_mag_r}), Table({"LUM_R": lums_r})])

In [ ]:
data_f[:5]

### Check the absolute and apparent magnitudes

This is a sanity check to quickly check whether there might be some obvious problem in the data/derivation of magnitudes...

In [ ]:
if len(data_f) < 1000000:
    # dauert sonst einfach ewig
    fig, ax = plt.subplots(ncols = 3, figsize=(10,5))

    plot_names = ["APP_MAG_R", "ABS_MAG_R", "LUM_R"]
    cnt = 0
    for axis in ax:
        axis.hist(data_f[plot_names[cnt]], bins = "auto")
        axis.set_yscale("log")
        cnt += 1
    plt.show()

In [ ]:
inspect_abs_mags = True

if inspect_abs_mags:
    data_to_inspect = data_f[data_f["ABS_MAG_R"]> -12]
    fig, ax = plt.subplots(ncols = 5, figsize = (15,8))
    
    ax[0].hist(data_to_inspect["ABS_MAG_R"], bins = "auto")
    ax[0].set_xlabel("abs mag in r-band")
    ax[1].hist(data_to_inspect["MW_TRANSMISSION"], bins = "auto")
    ax[1].set_xlabel("MW_TRANSMISSION")
    ax[2].hist(data_to_inspect["APP_MAG_R"], bins = "auto")
    ax[2].set_xlabel("App mag r-band")
    ax[3].hist(data_to_inspect["DIST_Z_MEAN"], bins = "auto")
    ax[3].set_xlabel("distance from redshift")
    ax[4].hist(data_to_inspect["Z"], bins = "auto")
    ax[4].set_xlabel("redshift")
    
    mean_redshift_pec_objects = np.mean(data_to_inspect["Z"])
    mean_distance_pec_objects = np.mean(data_to_inspect["DIST_Z_MEAN"])*u.Mpc
    min_redshift_pec_objects = np.min(data_to_inspect["Z"])
    min_distance_pec_objects = np.min(data_to_inspect["DIST_Z_MEAN"])*u.Mpc
    
    data_to_inspect_len = len(data_to_inspect)
    star_data_len = len(data_to_inspect[data_to_inspect["SPECTYPE"] == "STAR"])
    
    print(f"These objects have a mean redshift of {mean_redshift_pec_objects}, which corresponds to a mean distance of {mean_distance_pec_objects}")
    print(f"The minimum redshift is {min_redshift_pec_objects}, i.e. {min_distance_pec_objects}")
    print(f"According to WolframAlpha the closest galaxy has a distance of roughly 25000ly = 7655pc (not Mpc, simply pc), so any object below 0.07Mpc is definitely not a real galaxy")
    print(f"This reminded me to check if we have selected galaxies only...")
    print(f"We have {star_data_len} stars in our catalogue of peculiar objects, compared to a total number of {data_to_inspect_len} objects...")
                        
    for axis in ax:
        axis.set_ylabel("bin count")
        #axis.legend()
        
    plt.tight_layout()
    plt.show()
    

In [ ]:
# np.max(data_f["MW_TRANSMISSION"])

### start calculating the P_Gal (the probability that galaxy X is the host)

In [ ]:
max_lum_r = np.max(data_f["LUM_R"])*u.W
prob_dens_max_lum = data_f[data_f["LUM_R"] == max_lum_r]["PROBDENSITY"]
print(f"The maximum luminosity in our catalog in the r-band is {max_lum_r} with a prob density of {prob_dens_max_lum}")

In [ ]:
if modeled_search:

    """arg_errfc = abs(data_f["DIST_Z_MEAN"]-data_f["DISTMU"])/(data_f["DIST_Z_ERR"]**2+data_f["DISTSIGMA"]**2+data_f["DIST_Z_COSMO_SIGMA"].value**2)
    P_gal_unnorm = data_f["LUM_R"]*data_f["PROBDENSITY"]*(1-sc.special.erf(arg_errfc))
    #dist_mean*u.Mpc, dist_std*u.Mpc
    arg_errfc_c = abs(data_f["DIST_Z_MEAN"].value-dist_mean)/(data_f["DIST_Z_ERR"].value**2+dist_std**2+data_f["DIST_Z_COSMO_SIGMA"].value**2)
    P_gal_unnorm_const = data_f["LUM_R"]*data_f["PROBDENSITY"]*(1-sc.special.erf(arg_errfc_c))

    # now lets quickly normalize

    P_gal_sum = np.sum(P_gal_unnorm)
    P_gal_const_sum = np.sum(P_gal_unnorm_const)

    P_gal = 1/P_gal_sum * P_gal_unnorm
    P_gal_const = 1/P_gal_const_sum * P_gal_unnorm_const"""
    
    dist_delta = abs(data_f["DIST_Z_MEAN"]-data_f["DISTMU"])
    combined_sigma2 =  data_f["DISTSIGMA"]**2 + data_f["DIST_Z_COSMO_SIGMA"].value**2 + data_f["DIST_Z_ERR"]**2
    P_gal_unnorm = data_f["LUM_R"]*data_f["PROBDENSITY"]/(np.sqrt(2*np.pi*combined_sigma2))*np.exp(-dist_delta**2/(combined_sigma2))
    
    P_gal_sum = np.sum(P_gal_unnorm)
    P_gal = 1/P_gal_sum * P_gal_unnorm


else:
    P_gal_unnorm = data_f["LUM_R"]*data_f["PROBDENSITY"]
    P_gal_sum = np.sum(P_gal_unnorm)
    P_gal = 1/P_gal_sum * P_gal_unnorm

### Get the missed fraction of DESI due to mass/luminosity incompleteness

$$f_{missed} =  \frac{\int_{-\infty}^{\infty} dm 10^{-0.4m}\left(1-\frac{Hist}{Schechter}\right)\cdot Schechter}{\int_{-\infty}^{\infty}dm 10^{-0.4m}\cdot Schechter}$$

We want to account for the fact that some region were covered in more detail by DESI than others...

In [ ]:
if modeled_search:
    """
    # get the Schechter model, see 1/V estimator, bin size of half a mag
    from astropy.modeling.models import Schechter1D

    phi_star = 0.0016745140734036254
    m_star = -21.757361746831197
    alpha = -1.1297780395774693

    schechter_model = Schechter1D(phi_star, m_star, alpha)
    
    data_min_dist = np.min(data_f["DIST_Z_MEAN"])*u.Mpc
    data_max_dist = np.max(data_f["DIST_Z_MEAN"])*u.Mpc
    
    abs_mag_max = np.max(data_f["ABS_MAG_R"])
    abs_mag_min = np.min(data_f["ABS_MAG_R"])

    bins = np.arange(abs_mag_min, abs_mag_max, 0.5)

    hist, bins = np.histogram(data_f["ABS_MAG_R"], bins =  bins)
    
    # normalize our histogram to our volume
    volume = V_
    hist = hist/(volume)
    
    #hist = hist/(area_99.to_value(u.deg**2)*(data_max_dist-data_min_dist).value*10**0) # completely wrong normalization
    
    print(f"{abs_mag_min}, {abs_mag_max}, {data_min_dist}, {data_max_dist}, {(data_max_dist-data_min_dist).value*10**6}")
    
    fig, ax = plt.subplots(figsize = (5,2.5))
    bins_center = bins[1:]-(bins[1]-bins[2])/2
    ax.plot(bins_center, hist, label = "wrongly normalized distribution")
    ax.plot(np.linspace(-25,-18), schechter_model(np.linspace(-25,-18)),  label = "some schechter fit", color = "red")

    ax.set_yscale("log")
    plt.show()"""

In [ ]:
if modeled_search:
    def hist_func(M):
        # https://stackoverflow.com/questions/2566412/find-nearest-value-in-numpy-array
        # make sure array is a numpy array
        # array = np.array(array)
        idx = (np.abs(hist - M)).argmin()
        val = hist[idx]
        #print(f"val {val}")
        return val

    def num_func(M):
        #print(f"M is {M}")
        schechter_val = schechter_model(M)
        #print(f"schechter Val: {schechter_val}")
        lum_factor = 10**(-0.4*M)
        #print(f"lum factor {lum_factor}")
        weighting_coverage = (1-hist_func(M)/(schechter_val))
        #print(f"weighting_coverage: {weighting_coverage}")
        final_val = lum_factor*weighting_coverage*schechter_val
        #print(f"final value: {final_val}")
        return final_val
    def denom_func(M):
        lum_factor = 10**(-0.4*M)
        schechter_val = schechter_model(M)
        final_val = lum_factor*schechter_val

        return final_val




    numerator = quad(num_func, -28, -18)
    denominator = quad(denom_func, -28, -18)

    print(numerator)
    print(denominator)

    f_missed = numerator[0]/denominator[0]

    print(f_missed)

In [ ]:
data_f = hstack([data_f, Table({"P_GAL": P_gal.value})])

In [ ]:
data_f[:]

## Analyse the data: How much luminosity do we cover? Which galaxies should we observe?

Lets look into what information we can retrieve from here. First we define, how many galaxies we could observe (N_galaxies).

We then calculate the luminosity (lum) from all the galaxies in this area, i.e. the total luminosity. From there do some comparison between the covered luminosity and total luminosity depending on the ranking by probability:
- simple "Maximum lum that could be covered" by looking at the brightest galaxies
- Calculate the probability this way (done above) (from https://arxiv.org/pdf/1710.05452.pdf w/o normalization), both using a static and variable distance and error: $$P_{gal} = k^{-1}\tilde{L_z}(1-f_{missed})\cdot P_{2D}\left(1-\text{erf}\left(\frac{|D_{Gal}-D_{LVC}|}{\sqrt{\sigma_{D,gal}^{2}+\sigma_{D,LVC}^{2}+\sigma_{D,cosmology}^{2}}}\right)\right)$$
- others may follow...

**Wrong formula**

Since we have the actual luminosity (here in r-band though), we don't need to again calculate the $$\tilde{L}_{gal}$$ values. In the mentioned paper, they only use a fixed distance and error for the LIGO data. I will try both, to get an idea what the differences are.

In [ ]:
N_galaxies = 200 # how many galaxies we can cover

In [ ]:
total_lum = np.sum(data_f["LUM_R"])

### Compare lum with luminosity ranking

https://iopscience.iop.org/article/10.3847/0067-0049/226/1/10

In [ ]:
data_sort_by_lums = data_f.group_by("LUM_R")
data_sort_by_lums = data_sort_by_lums[::-1]

In [ ]:
lum_after_N_highest = np.sum(data_sort_by_lums["LUM_R"][:N_galaxies])

In [ ]:
print(f"The theoretical maximum luminosity we could cover is {lum_after_N_highest}, i.e. {lum_after_N_highest*100/total_lum} % of the total luminosity")
print("The TARGETID values of these galaxies are: \n", data_sort_by_lums["TARGETID"][:N_galaxies])

### Compare lum with P_Gal ranking

In [ ]:
data_sort_by_PGal = data_f.group_by("P_GAL")
data_sort_by_PGal = data_sort_by_PGal[::-1]

lum_after_N_PGal = np.sum(data_sort_by_PGal["LUM_R"][:N_galaxies])

In [ ]:
print(f"The luminosity we cover when observing {N_galaxies} galaxies ranked by P_gal is {lum_after_N_PGal}W, i.e. {lum_after_N_PGal*100/total_lum}% of the total (DESI) luminosity in the search area") 
print("The TARGETID values of these galaxies are: \n", data_sort_by_PGal["TARGET_RA", "TARGET_DEC", "SERSIC", "TARGETID", "P_GAL"][:N_galaxies])

In [ ]:
P_Gal_cumsum = np.cumsum(data_sort_by_PGal["P_GAL"])

In [ ]:
plt.plot(list(range(len(P_Gal_cumsum))),  P_Gal_cumsum)
plt.xscale("log")

plt.yscale("log")
plt.xlim(1,len(P_Gal_cumsum))
plt.savefig(dpi = 100, fname = f"{superevent_name}PGAL_CUMULATIVE099.png")
plt.show()

In [ ]:
"""data_sort_by_PGal_C = data_f.group_by("P_GAL_C")
data_sort_by_PGal_C = data_sort_by_PGal_C[::-1]

lum_after_N_PGal_C = np.sum(data_sort_by_PGal_C["LUM_R"][:N_galaxies])"""

In [ ]:
"""print(f"The maximum luminosity we cover when ranked by P_gal_C is {lum_after_N_PGal_C}, i.e. {lum_after_N_PGal_C*100/total_lum}% of the total luminosity") 
print("The TARGETID values of these galaxies are: \n", data_sort_by_PGal_C["TARGETID", "P_GAL"][:N_galaxies])"""

## Exclude unobservabe objects from the PGal ranking and export list

We want to cut out galaxies that are not observable by Wendelstein. For this we need the date (to calculate alt and az; we do this for three times to check observability) and Wendelstein location.

In [ ]:
# convert DEG:MIN:SEC to deg with decimals...
def conv_to_deg(deg, arcmin, arcsec):
    return (deg+ arcmin/60+ arcsec/3600)*u.deg

### Get the observable objects (i.e. alt > 0) in the next night and apply that selection to our data

In [ ]:
# set the timzone
timezone = "Europe/Berlin"
tz_Berlin = pytz.timezone(timezone)

# get the date (and actually time, but that it useless for us) of today and tomorrow
day_Berlin = datetime.now(tz_Berlin)
tmrw_Berlin = day_Berlin + timedelta(hours = 23, minutes = 59, seconds = 59)

# set the Wendelstein location
Wendelstein_loc = EarthLocation(lat = conv_to_deg(47, 42, 13.1), lon = conv_to_deg(12, 0, 43.4), height = 1838*u.m) #check - or + 12

# define the sun object, which will tell us sunrise and set
sun = SunTimes(Wendelstein_loc.lon.to_value(), Wendelstein_loc.lat.to_value(), Wendelstein_loc.height.to_value())
sunset_time = sun.setwhere(day_Berlin, timezone)
sunrise_time = sun.risewhere(tmrw_Berlin, timezone)

# get a time array that includes the sunset time and from there a 30min interval until sunset
t = np.arange(sunset_time, sunrise_time, timedelta(minutes=45)).astype(datetime)              # this may throw a deprecation warning, but seems fine anyway: https://github.com/numpy/numpy/issues/23904

# a little cheap but effective: observation at sunset is useless, start 1h after:
t = t[2:]

In [ ]:
# get all the SkyCoords for each objects in the data catalog
target_SkyCoord = SkyCoord(data_sort_by_PGal["TARGET_RA"], data_sort_by_PGal["TARGET_DEC"], unit = "deg")

In [ ]:
# get Alt for the different times during the night
# t should be an array that consists of a couple of night times, so if *all* of these times don't have a positive alt it will be discarded
target_Alt = [target_SkyCoord.transform_to(AltAz(obstime = time, location = Wendelstein_loc)).alt.value for time in t]

# now turn it into a numpy array
target_Alt = np.array(target_Alt)

# swap the axes so that the first axis is all the different targets and the second then includes the alt per observing time
target_Alt_swapped = np.swapaxes(target_Alt, 0,1)

# now create the selection for each target in the P_Gal catalog
sel = []
for target in target_Alt_swapped:
    sel.append((target > 0).any())
    
# now apply the selection to the dataset
data_sort_by_PGal_observable = data_sort_by_PGal[sel]

In [ ]:
print(f"There were {len(data_sort_by_PGal) - len(data_sort_by_PGal_observable)} non-observable objects in the current dataset. \nWe have {len(data_sort_by_PGal_observable)} objects remaining")

### Export the final list & print some data

In [ ]:
print(f"These are the stats for {superevent_name} follow up :")

In [ ]:
data_min_z = np.min(data_sort_by_PGal_observable["Z"])
data_max_z = np.max(data_sort_by_PGal_observable["Z"])
print(f"Localization area: {area_99.to_value(u.deg**2)*u.deg**2} \nRedshift range in data: {data_min_z} to {data_max_z} \ncorresponding lum distance range: {data_min_dist} to {data_max_dist} \nhighest prob location is at RA = {ra.deg*u.deg} and dec = {dec.deg*u.deg} \ntotal No. of objects in desi data is {len(data_sort_by_PGal)}")


In [ ]:
data_sort_by_PGal_observable["TARGET_RA", "TARGET_DEC", "SERSIC", "Z", "P_GAL"][:N_galaxies].write(f'PGAL/PGAL_{superevent_name}.ecsv', delimiter=',', format='ascii', overwrite = True)


In [ ]:
# add timing data to table, 